In [2]:
# Import Node and Workflow object and FSL interface
import numpy as np
from nipype import Node, Workflow, IdentityInterface
from os.path import abspath
import pycartool as cart
import mne
import pandas as pd


import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
from traits.api import *
import sys
sys.path.append('/Volumes/SSD-Bercows/Joan/research/2_projects/connectomemapper3/')
sys.path.append('/Volumes/SSD-Bercows/Joan/research/2_projects/connectomemapper3/dev/')

from cmp.pipelines.common import *
import cmp.pipelines.common as cmp_common
from cmp.project import CMP_Project_Info
import datetime
from cmp_dev.stages.eeg.eeg_loader import EEGLoaderStage
from cmp_dev.stages.eeg.eeg_preprocessing import EEGPreprocessingStage
from cmp_dev.stages.eeg.eeg_inverse_solution import EEGInverseSolutionStage

from cmtklib_dev.interfaces import invsol, eeglab2fif, createrois, eegloader

print('Done importing!')

210317-13:43:51,169 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0
Done importing!


In [3]:
class Global_Configuration(HasTraits):

    
    process_type = Str('EEG')
    subjects = List(trait=Str)
    subject = Str
    subject_session = Str
   
    def __init__(self,base_dir):
        self.base_directory = base_dir
    

class EEGPipeline(Pipeline):
    
    pipeline_name = Str("EEG_pipeline")
    input_folders = ['anat','eeg']    
    
    
    def __init__(self,project_info):
        self.global_conf = Global_Configuration(project_info.base_directory)
        self.subject_id = project_info.subject_id
        self.subject = project_info.subject
        self.global_conf.subject_session = ''
        self.subject_directory = os.path.join(project_info.base_directory,
                                                  project_info.subject)
        self.stages = {'EEGPreprocessing': EEGPreprocessingStage(pipeline_mode='EEGLAB',
                                                                 bids_dir=project_info.base_directory,
                                                                 output_dir=self.output_directory),
                       'EEGInverseSolution': EEGInverseSolutionStage(pipeline_mode='CARTOOL',
                                                                     bids_dir=project_info.base_directory,
                                                                     output_dir=self.output_directory),
                       }
        cmp_common.Pipeline.__init__(self, project_info)

        
bids_dir = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/'
output_dir = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg_output/'
participant_label = '01'

project = CMP_Project_Info()
project.base_directory = os.path.abspath(bids_dir)
project.output_directory = os.path.abspath(output_dir)
project.subjects = ['{}'.format(participant_label)]
project.subject = '{}'.format(participant_label)
project.subject_id = '{}'.format(participant_label)
project.subject_sessions = ['']
project.subject_session = ''

self = EEGPipeline(project)
cmp_deriv_subject_directory = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/cmp-v3.0.0-beta-RC1/'
nipype_deriv_subject_directory = '/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/nipype/'

datasource = pe.Node(eegloader.EEGLoader(), name = "eegloader")
datasource.inputs.base_directory = self.base_directory
datasource.inputs.subject_id = self.subject_id

# Data sinker for output
sinker = pe.Node(nio.DataSink(), name="eeg_sinker")
sinker.inputs.base_directory = os.path.abspath(cmp_deriv_subject_directory)

# Define nodes
eeglab2fif_node = pe.Node(eeglab2fif.EEGLAB2fif(), name="eeglab2fif")
is_node = pe.Node(invsol.CartoolInverseSolutionROIExtraction(), name="invsolution")
createrois_node = pe.Node(createrois.CreateRois(),"create_rois")

 # Clear previous outputs
self.clear_stages_outputs()

# Create common_flow
eeg_flow = pe.Workflow(name='eeg_pipeline', base_dir=os.path.abspath(
    nipype_deriv_subject_directory))

eeg_inputnode = pe.Node(interface=util.IdentityInterface(
                    fields=['epochs', 
                            'src_file', 
                            'invsol_file', 
                            'lamda', 
                            'svd_params',
                            'cartool_dir',
                            'cmp3_dir',
                            'subject_id',
                            'parcellation',
                            'behav_file',
                            'epochs_fif_fname',
                            'roi_ts_file',
                            'base_dir',
                            ], 
                    mandatory_inputs=True), name="inputnode")

eeg_inputnode.inputs.base_dir = self.global_conf.base_directory
eeg_inputnode.inputs.lamda = 6
eeg_inputnode.inputs.svd_params =  {'toi_begin' : 120,
                                    'toi_end' : 500}

eeg_inputnode.inputs.cartool_dir = os.path.join(self.base_directory,'derivatives','cartool')
eeg_inputnode.inputs.cmp3_dir = os.path.join(self.base_directory,'derivatives','cmp-v3.0.0-beta-RC1')
eeg_inputnode.inputs.subject_id = self.subject_id
eeg_inputnode.inputs.epochs_fif_fname = os.path.join(self.base_directory,'derivatives','cmp-v3.0.0-beta-RC1','sub-'+self.subject_id,'eeg','sub-'+self.subject_id+'_epo.fif')
eeg_inputnode.inputs.roi_ts_file = os.path.join(self.base_directory,'derivatives','cmp-v3.0.0-beta-RC1','sub-'+self.subject_id,'eeg','sub-'+self.subject_id+'_rtc_epo.npy')

eeg_outputnode = pe.Node(interface=util.IdentityInterface(
                    fields=["roi_ts_file"]), name="outputnode")

eeg_flow.add_nodes([eeg_inputnode, eeg_outputnode])

# 0

eeg_flow.connect([
    (datasource, eeg_inputnode, 
     [("EEG", "epochs"),
      ('events','behav_file'),
      ('src_file','src_file'),
      ('invsol_file','invsol_file'),
      ('parcellation','parcellation'),
     ]),
])

# 1
eeg_flow.connect([(eeg_inputnode, eeglab2fif_node,
                 [('epochs','eeg_ts_file'),
                  ('behav_file','behav_file'),
                  ('epochs_fif_fname','epochs_fif_fname'),
                 ]
    )])

# 2
eeg_flow.connect([(eeglab2fif_node, is_node,
                 [('fif_ts_file','eeg_ts_file'),
                 ]
    )])

# 3
eeg_flow.connect([(eeg_inputnode, is_node,
                 [('src_file','src_file'),
                  ('invsol_file','invsol_file'),
                  ('lamda','lamda'),
                  ('svd_params','svd_params'),
                  ('roi_ts_file','roi_ts_file'),
                 ]
    )])

# 4
eeg_flow.connect([(eeg_inputnode, createrois_node,
                 [('subject_id','subject_id'),
                  ('parcellation','parcellation'),
                  ('cartool_dir','cartool_dir'),
                  ('cmp3_dir','cmp3_dir'),
                 ]
    )])
# 5
eeg_flow.connect([(createrois_node, is_node,
                 [('rois_pickle','rois_file'),
                 ]
    )])       
# 6
eeg_flow.connect([(is_node, eeg_outputnode,
                 [('roi_ts_file','roi_ts_file'),
                 ]
    )])

In [4]:
eeg_flow.run()

210317-13:43:53,514 nipype.workflow INFO:
	 Workflow eeg_pipeline settings: ['check', 'execution', 'logging', 'monitoring']
210317-13:43:53,527 nipype.workflow INFO:
	 Running serially.
210317-13:43:53,527 nipype.workflow INFO:
	 [Node] Setting-up "eeg_pipeline.eegloader" in "/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/nipype/eeg_pipeline/eegloader".
210317-13:43:53,539 nipype.workflow INFO:
	 [Node] Running "eegloader" ("cmtklib_dev.interfaces.eegloader.EEGLoader")
210317-13:43:54,225 nipype.workflow INFO:
	 [Node] Finished "eeg_pipeline.eegloader".
210317-13:43:54,227 nipype.workflow INFO:
	 [Node] Setting-up "eeg_pipeline.create_rois" in "/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/nipype/eeg_pipeline/create_rois".
210317-13:43:54,237 nipype.workflow INFO:
	 [Node] Running "create_rois" ("cmtklib_dev.interfaces.createrois.CreateRois")
210317-13:45:10,93 nipype.workflow INFO:
	 [Node] Finished "eeg_pipeline.create_rois".
21031

In [6]:
eeg_flow.write_graph(graph2use='colored',
                              format='png', simple_form=True)

210317-14:08:38,922 nipype.workflow INFO:
	 Generated workflow graph: /Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/nipype/eeg_pipeline/graph.png (graph2use=colored, simple_form=True).


'/Volumes/SSD-Bercows/Joan/research/0_data/DS001_BIDS_cmp-eeg/derivatives/nipype/eeg_pipeline/graph.png'